In [ ]:
from google.colab import drive
drive.mount('/content/driver/')

Mounted at /content/driver/


In [ ]:
%cd driver/My\ Drive/dip

/content/driver/My Drive/dip


In [ ]:
!git clone https://github.com/zb46392/poker.git

Cloning into 'poker'...
remote: Enumerating objects: 457, done.
remote: Counting objects: 100% (457/457), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 457 (delta 253), reused 369 (delta 167), pack-reused 0
Receiving objects: 100% (457/457), 107.38 KiB | 1.85 MiB/s, done.
Resolving deltas: 100% (253/253), done.


In [ ]:
%cd poker

/content/driver/My Drive/dip/poker


In [ ]:
from itertools import product
from game import TrainingTable, OpponentBot
from game.table.observer import FileTextualObserver
from typing import Dict, Tuple
from random import shuffle

In [ ]:
def generate_parameters():
    d = dict(
        fold_thresh=[10, 20, 30, 40, 50, 60, 70, 80, 90],
        raise_thresh=[10, 20, 30, 40, 50, 60, 70, 80, 90],
        bluff_thresh=[10, 20, 30, 40, 50, 60, 70, 80, 90]
    )

    return d

In [ ]:
def init_table(group: Tuple[Tuple[int, int, int], Tuple[int, int, int], Tuple[int, int, int]]) -> TrainingTable:
    t = TrainingTable([OpponentBot, OpponentBot, OpponentBot])

    p = t.players

    for i in range(len(p)):
        p[i].fold_thresh = group[i][0]
        p[i].raise_thresh = group[i][1]
        p[i].bluff_thresh = group[i][2]

    return t

In [ ]:
def create_player_param(t: TrainingTable) -> Dict[str, Dict[str, int]]:
    p = t.players
    pn = t.player_names

    param = {}

    for i in range(len(p)):
        param[pn[i]] = {'fold': p[i].fold_thresh, 'raise': p[i].raise_thresh, 'bluff': p[i].bluff_thresh}

    return param

In [ ]:
parameters = generate_parameters()
parameters = parameters.values()
combos = [(fold_thresh, raise_thresh, bluff_thresh) for fold_thresh, 
          raise_thresh, bluff_thresh in product(*parameters)]
shuffle(combos)

while len(combos) != 3:
    groups = [(combos[i], combos[i + 1], combos[i + 2]) for i in range(0, len(combos), 3)]
    winners = []

    for g in groups:
        t = init_table(g)
        player_params = create_player_param(t)
        t.run_tournament()
        winner = t.get_winner_name()
        win_params = player_params[winner]

        winners.append((win_params['fold'], win_params['raise'], win_params['bluff']))
        t.reset_tournament()

    combos = winners

t = init_table(combos)
print('Finale:')
for p in t.players:
    print(f'{p.name} -> fold: {p.fold_thresh}, raise: {p.raise_thresh}, bluff: {p.bluff_thresh}')

t.attach_observer(FileTextualObserver())
t.run_tournament()
print(f'Winner: {t.get_winner_name()}')

Finale:
Player_1 (OpponentBot) -> fold: 70, raise: 50, bluff: 20
Player_2 (OpponentBot) -> fold: 60, raise: 80, bluff: 30
Player_3 (OpponentBot) -> fold: 50, raise: 50, bluff: 70
Winner: Player_2 (OpponentBot)
